In [1]:
from handcalcs import render
import forallpeople
forallpeople.environment("structural", top_level=True)

from reinforced_concrete.sections import create_concrete_material, create_steel_material, Bars, ReinforcedConcreteSection, InternalForces, Stirrups
from reinforced_concrete.sls import sls

### Quasi permanente

In [ ]:
cls  = create_concrete_material("EC2","C25/30") 
steel  = create_steel_material("NTC18","B450C")

sec = ReinforcedConcreteSection(
    b=300, 
    d=460, 
    d1=40, 
    d2=40, 
    concrete_material=cls, 
    As=Bars(n_bars=4, diameter=18, steel_material=steel), 
    As1=Bars(n_bars=2, diameter=18, steel_material=steel), 
    stirrups=Stirrups(n_braces=2, diameter=8, spacing=220, alpha=90),
    internal_forces=InternalForces(M=79.04*10**6, N=0.), 
    name="C5_QP"
    )
cls.is_sls_qp = True   

res_sec = sls(section=sec)[0]
res_sec

{'x': 157.50801020071052,
 'sigma_c': 6.575929515967569,
 'sigma_s': 189.43474698811877,
 'sigma_s1': 73.58905667624114,
 '<sigma_cR': True,
 '<sigma_sR': True,
 '<sigma_sR1': True}

In [ ]:
%%render 2 short
E_s = ((sec.As.steel_material.Es*MPa).prefix("M"))
E_cm = ((sec.concrete_material.Ecm*MPa).prefix("M"))
f_ctm = ((sec.concrete_material.fctm*MPa).prefix("M"))
h = (sec.h*mm)
d = (sec.d*mm)
b = (sec.b*mm)
A_s = (sec.As.area*mm**2)
Phi = (sec.As.diameter*mm) #Nel caso le barre abbiano diametri diversi, occorre calcolarne uno equivalente

<IPython.core.display.Latex object>

In [ ]:
%%render 2 short
x = (res_sec["x"]*mm)
sigma_s = (res_sec["sigma_s"]*MPa)
alpha_e = E_s/E_cm 
h_c_eff = min(2.5*(h-d),(h-x)/3,h/2)
A_c_eff = b*h_c_eff
rho_eff = A_s/A_c_eff


k_t = 0.4 #Quasi permanente
epsilon_sm_QP = (sigma_s - k_t*f_ctm/rho_eff * (1+alpha_e*rho_eff))/E_s
epsilon_sm_lim_QP = 0.6*sigma_s/E_s

epsilon_sm_vero_QP = max(epsilon_sm_QP, epsilon_sm_lim_QP)

<IPython.core.display.Latex object>

In [ ]:
%%render 3
## Scegliere k1, k2 e c qui sotto:

k_1 = 0.8 # barre ad aderenza migliorata
k_2 = 0.5 # nel caso della flessione
k_3 = 3.4 # fisso
k_4 = 0.425 # fisso

c = (20*mm) # ricoprimento barre
Delta_sm = (k_3*c + k_1*k_2*k_4*Phi/rho_eff)/1.7

w_k_QP = (1.7*epsilon_sm_vero_QP*Delta_sm).prefix("m")

<IPython.core.display.Latex object>

## Frequente

In [ ]:
sec.internal_forces = InternalForces(M=102.61*10**6, N=0.)
sec.name = "Freq"
cls.is_sls_qp = False   

res_sec = sls(section=sec)[0]
res_sec

{'x': 157.50801020071052,
 'sigma_c': 8.536894327346056,
 'sigma_s': 245.924840440927,
 'sigma_s1': 95.5335666187893,
 '<sigma_cR': True,
 '<sigma_sR': True,
 '<sigma_sR1': True}

In [ ]:
%%render 3 short
## Analogamente a quanto fatto sopra, con una sigmas di combinazione frequente si ha ora:
x = (res_sec["x"]*mm)
sigma_s = (res_sec["sigma_s"]*MPa)

k_t = 0.6 #Frequente
epsilon_sm_FREQ = (sigma_s - k_t*f_ctm/rho_eff * (1+alpha_e*rho_eff))/E_s
epsilon_sm_lim_FREQ = 0.6*sigma_s/E_s
epsilon_sm_vero_FREQ = max(epsilon_sm_FREQ, epsilon_sm_lim_FREQ)

w_k_freq = (1.7*epsilon_sm_vero_FREQ*Delta_sm).prefix("m")

<IPython.core.display.Latex object>